In [28]:
# punket

In [60]:
import nltk.data
import pandas as pd
from nltk.tokenize import PunktSentenceTokenizer
from sklearn.metrics import confusion_matrix, classification_report, f1_score
import time

# Import the Punkt tokenizer from the nltk.data module
nltk.data.path.append("nltk_data")  # Specify the path to the nltk_data directory if necessary
custom_sent_tokenize = nltk.data.load('tokenizers/punkt/english.pickle')

# Read the contents of the file
file_path = "europarl_en.txt"
with open(file_path, 'r', encoding='utf-8') as file:
    text = file.read()

# Remove newline characters and tab characters
text = text.replace('\\r', '').replace('\\n', '').replace('\\t', '')

# Define custom sentence boundaries
custom_sent_tokenize._params.abbrev_types.update(['.', ',', ';', 'e.g.', 'i.e.', 'etc.', 'mr.', 'mrs.', 'ms.', 'dr.'])
custom_sent_tokenize._params.abbrev_types.update(['v.', 'vs.', 'art.', 'no.', 'jr.', 'sr.', 'inc.', 'co.'])

# Measure the time taken for sentence boundary detection
start_time = time.time()

# Tokenize the text into sentences
predicted_sentences = custom_sent_tokenize.tokenize(text)

# Measure the elapsed time
elapsed_time = time.time() - start_time

# Read the labeled dataset for evaluation
labeled_dataset_path = 'sentences_labels.csv'
df = pd.read_csv(labeled_dataset_path)

# Get the expected sentences and labels from the labeled dataset
expected_sentences = df['Sentence'].tolist()
expected_labels = df['Label'].tolist()

# Filter out sentences from the labeled dataset that are not present in the predicted sentences
filtered_expected_labels = []
filtered_expected_sentences = []
for sent, label in zip(expected_sentences, expected_labels):
    if sent in predicted_sentences:
        filtered_expected_sentences.append(sent)
        filtered_expected_labels.append(label)

# Create a list to store the predicted labels
predicted_labels = []
for sent in filtered_expected_sentences:
    idx = predicted_sentences.index(sent)
    predicted_labels.append(expected_labels[idx])

# Calculate accuracy
correct_predictions = sum(1 for true_label, pred_label in zip(filtered_expected_labels, predicted_labels) if true_label == pred_label)
total_predictions = len(filtered_expected_labels)
accuracy = correct_predictions / total_predictions

# Generate classification report
report = classification_report(filtered_expected_labels, predicted_labels)
print("Classification Report:")
print(report)

# Calculate confusion matrix
confusion_mat = confusion_matrix(filtered_expected_labels, predicted_labels)
print("Confusion Matrix:")
print(confusion_mat)

# Calculate F1 score
f1 = f1_score(filtered_expected_labels, predicted_labels, average='weighted')
print("F1 Score:", f1)

# Print accuracy and elapsed time
print("Accuracy:", accuracy)
print("Elapsed Time:", elapsed_time, "seconds")


Classification Report:
              precision    recall  f1-score   support

           0       0.00      0.00      0.00         1
           1       1.00      0.51      0.68      3110

    accuracy                           0.51      3111
   macro avg       0.50      0.25      0.34      3111
weighted avg       1.00      0.51      0.68      3111

Confusion Matrix:
[[   0    1]
 [1524 1586]]
F1 Score: 0.6751075989263068
Accuracy: 0.5098039215686274
Elapsed Time: 0.16756105422973633 seconds


In [44]:
import nltk.data
from nltk.tokenize import PunktSentenceTokenizer

# Import the Punkt tokenizer from the nltk.data module
nltk.data.path.append("nltk_data")
custom_sent_tokenize = nltk.data.load('tokenizers/punkt/english.pickle')

# Define your custom sentence
#custom_sentence = "This is a custom sentence. It has multiple sentences. Can the model detect the boundaries?"

with open('sample.txt', 'r') as file:
    custom_sentence = file.read()

# Tokenize the custom sentence into sentences
predicted_sentences = custom_sent_tokenize.tokenize(custom_sentence)

# Print the separated sentences
i=0
for sentence in predicted_sentences:
    i=i+1
    print(i, sentence)


1 Mr. Johnson, the CEO of XYZ Corp., arrived at the meeting at 9:00 a.m. with his wife, Mrs. Johnson!
2 They were greeted by the board members, including Dr. Anderson, the company's Chief Financial Officer, and Ms. Thompson, the Head of Marketing?
3 The meeting room was adorned with elegant decorations and state-of-the-art technology!
4 During the meeting, they discussed various topics, including financial strategies, marketing campaigns, and employee engagement.
5 They analyzed the current market trends, reviewed sales reports, and brainstormed innovative ideas.
6 It was a high-energy session with everyone actively participating and sharing their perspectives!
7 The meeting lasted for hours, and they covered a wide range of issues.
8 They deliberated on cost-cutting measures, expansion plans, and investment opportunities.
9 They also addressed concerns raised by the staff, such as work-life balance and career development!
10 The atmosphere was intense yet collaborative, with each pers

In [ ]:
#crf

In [47]:
import nltk
import sklearn_crfsuite
from sklearn.metrics import multilabel_confusion_matrix, classification_report, accuracy_score
from sklearn import metrics
from sklearn.preprocessing import MultiLabelBinarizer
import time


# Defining Feature and Label Functions
def extract_features(sentence, i):
    features = {
        'word': sentence[i],
        'is_first': i == 0,
        'is_last': i == len(sentence) - 1,
        'is_capitalized': sentence[i][0].upper() == sentence[i][0],
        'is_all_caps': sentence[i].upper() == sentence[i],
        'is_all_lower': sentence[i].lower() == sentence[i],
        'is_alphanumeric': int(bool((len(sentence[i]) > 1) & sentence[i].isalnum()))
    }
    return features


def get_label(sentence, i):
    last_char = sentence[i][-1]
    return str(last_char == ',' or last_char == ';' or last_char == ':' or last_char == '.' or last_char == '?')


# read text from file
with open('europarl_en.txt', 'r', encoding='utf-8') as f:
    text = f.read()

text = text.replace('\\r', '').replace('\\n', '').replace('\\t', '')

# perform sentence boundary detection
sentences = nltk.sent_tokenize(text)
X = [nltk.word_tokenize(sentence) for sentence in sentences]
y = [[get_label(sentence, i) for i in range(len(sentence))] for sentence in X]

# extract features from sentences
X_feats = [[extract_features(X[i], j) for j in range(len(X[i]))] for i in range(len(X))]

# Convert labels to list of sequences
y_seq = [sentence_labels for sentence_labels in y]

start_time = time.time()

# train CRF model
crf = sklearn_crfsuite.CRF(algorithm='lbfgs', c1=0.1, c2=0.1, max_iterations=3)
crf.fit(X_feats, y_seq)

elapsed_time = time.time() - start_time
print("elapsed time",elapsed_time)

# test CRF model on training data
mlb = MultiLabelBinarizer()
y_bin = mlb.fit_transform(y_seq)

# test CRF model on training data
y_pred = crf.predict(X_feats)

# Convert y_pred to binary array format (if needed)
y_pred_bin = mlb.transform(y_pred)

# Generate classification report
report = classification_report(y_bin, y_pred_bin)
print(report)

# Calculate confusion matrix
confusion_matrix = multilabel_confusion_matrix(y_bin, y_pred_bin)
print(confusion_matrix)

# Calculate accuracy
accuracy = accuracy_score(y_bin, y_pred_bin)
print('Accuracy:', accuracy)

# Calculate F1 score
f1_score = metrics.f1_score(y_bin, y_pred_bin, average='weighted')
print('F1 score:', f1_score)

# Print boundary-detected text
for sentence, labels in zip(X, y_pred):
    boundary_detected_text = " ".join([token for token, label in zip(sentence, labels)])

elapsed time 1.0521612167358398
              precision    recall  f1-score   support

           0       1.00      1.00      1.00      3244
           1       1.00      1.00      1.00      3244

   micro avg       1.00      1.00      1.00      6488
   macro avg       1.00      1.00      1.00      6488
weighted avg       1.00      1.00      1.00      6488
 samples avg       1.00      1.00      1.00      6488

[[[   7    0]
  [   0 3244]]

 [[   0    7]
  [   0 3244]]]
Accuracy: 0.9978468163641956
F1 score: 0.9994611239414934


In [45]:
import nltk
import sklearn_crfsuite
from sklearn.preprocessing import MultiLabelBinarizer

# Custom sentence
#custom_sentence = "This is a custom sentence. It has multiple sentences. Can the model detect the boundaries?"
with open('sample.txt', 'r') as file:
    custom_sentence = file.read()

# Tokenize the custom sentence
custom_tokens = nltk.word_tokenize(custom_sentence)

# Extract features from the custom sentence
custom_features = [extract_features(custom_tokens, i) for i in range(len(custom_tokens))]

# Predict the labels for the custom sentence
custom_labels = crf.predict([custom_features])[0]

# Combine tokens based on the predicted labels
separated_sentences = []
current_sentence = []
for token, label in zip(custom_tokens, custom_labels):
    current_sentence.append(token)
    if label == 'True':  # Sentence boundary detected
        separated_sentences.append(" ".join(current_sentence))
        current_sentence = []

# Add the last sentence if not already added
if current_sentence:
    separated_sentences.append(" ".join(current_sentence))

# Print the separated sentences
i=0
for sentence in separated_sentences:
    i=i+1
    print(i, sentence)


1 Mr. Johnson ,
2 the CEO of XYZ Corp. ,
3 arrived at the meeting at 9:00
4 a.m. with his wife ,
5 Mrs. Johnson !
6 They were greeted by the board members ,
7 including Dr. Anderson ,
8 the company 's Chief Financial Officer ,
9 and Ms. Thompson ,
10 the Head of Marketing ?
11 The meeting room was adorned with elegant decorations and state-of-the-art technology !
12 During the meeting ,
13 they discussed various topics ,
14 including financial strategies ,
15 marketing campaigns ,
16 and employee engagement .
17 They analyzed the current market trends ,
18 reviewed sales reports ,
19 and brainstormed innovative ideas .
20 It was a high-energy session with everyone actively participating and sharing their perspectives !
21 The meeting lasted for hours ,
22 and they covered a wide range of issues .
23 They deliberated on cost-cutting measures ,
24 expansion plans ,
25 and investment opportunities .
26 They also addressed concerns raised by the staff ,
27 such as work-life balance and car